In [1]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from keras.utils import pad_sequences
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import io
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from langchain.text_splitter import TokenTextSplitter
import nltk
# nltk.download('punkt')  # Download the punkt tokenizer for sentence tokenization
from nltk.tokenize import sent_tokenize

2023-07-20 13:59:26.928860: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-20 13:59:26.984370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 13:59:27.708169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

# Load the saved state dictionary into the model
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.load_state_dict(torch.load(model_file))
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)


cuda:2


In [3]:
device

device(type='cuda', index=2)

In [7]:

def get_first_sentence_prediction(model,tokenizer, sentences):
    # Load the BERT model and tokenizer
    # model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    # tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # We need to add special tokens at the beginning and end of each sentence
    # for BERT to work properly
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
    labels = [0]  # Dummy labels for prediction, you can ignore this since we only predict the first sentence.

    # Tokenize and pad the sentences
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = pad_sequences(
        [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
        maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert inputs to tensors
    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    prediction_labels = torch.tensor(labels)

    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Store predictions and true labels
    predictions = np.argmax(logits, axis=1).flatten()

    return predictions[0]



        
# # Example usage
# sentences = ["They drank the pub.", "She likes ice cream."]
# predicted_label = get_first_sentence_prediction(sentences)
# print(predicted_label)


In [14]:

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        for page in PDFPage.get_pages(file, check_extractable=True):
            page_interpreter.process_page(page)
            text += fake_file_handle.getvalue()
        
        converter.close()
        fake_file_handle.close()
    
    return text
# Function to divide text into chunks of 512 tokens
def divide_text_into_chunks(text, max_chunk_size=512):
    tokenized_texts = text.split()  # Assuming the text is already tokenized by spaces, adjust as needed
    chunks = []
    current_chunk = ""
    
    for tokenized_text in tokenized_texts:
        if len(current_chunk) + len(tokenized_text) <= max_chunk_size:
            current_chunk += tokenized_text + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = tokenized_text + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks
text_splitter_1 = TokenTextSplitter(chunk_size=100,chunk_overlap = 2)
# Example usage
pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = text_splitter_1.split_text(text)
# chunks = divide_text_into_chunks(text)
# Predict each chunk
for idx, chunk in enumerate(chunks[:1]):
    sentences = sent_tokenize(chunk)
    label_mapping = {0: "incorrect", 1: "correct"}
    for sentence in sentences:
        sentence_label = get_first_sentence_prediction(model, tokenizer, [sentence])
        print(f"Chunk {idx+1} -  Sentence {sentences.index(sentence) + 1}: Sentence Label: {label_mapping[sentence_label]}, {sentence}")

Sentence 1 in Chunk 1: Sentence Label: incorrect, Learner Choice, Learning Voice 

Learner Voice, Learner Choice offers fresh, forward-thinking supports for teachers creating 
an empowered, student-centered classroom.
Sentence 2 in Chunk 1: Sentence Label: correct, Learner agency is a major topic in today’s 
schools, but what does it mean in practice, and how do these practices give students skills and 
opportunities they will need to thrive as citizens, parents, and workers in our ever-shifting 
climate?
Sentence 3 in Chunk 1: Sentence Label: incorrect, Showcasing


In [11]:
# import torch
# from transformers import BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# Tokenize Inputs

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)
# Replace 'path_to_model_file.pth' with the path to the saved .pth model file.
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'

# Create an instance of the model's class
# model = BertForSequenceClassification()

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=2)
# Load the saved state dictionary into the model
model.load_state_dict(torch.load(model_file))

# Optional: If you want to use the model for inference, move it to the appropriate device (GPU or CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# run on a sample text

model.eval()

# Create sentence) and label lists
sentences = ["They drank the pub."]
# We need to add special tokens at the beginning and end of each sentence
# for BERT to work properly
print("sentences")
print(sentences)
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels =[0]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("tokenized_texts")
print(tokenized_texts)
# Padding Sentences
# Set the maximum sequence length. The longest sequence in our training set
# is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 128
predictions = []
# Pad our input tokens
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
    maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
# Index Numbers and Padding
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# pad sentences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype ="long", truncating="post",padding ="post")
# Attention masks
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i > 0) for i in seq]
  attention_masks.append(seq_mask)
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Put model in evaluation mode
model.eval()
with torch.no_grad():
  # Forward pass, calculate logit predictions
  logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
# Move logits and labels to CPU
logits = logits.detach().cpu().numpy()
# label_ids = b_labels.to("cpu").numpy()
# Store predictions and true labels
predictions.append(logits)
# true_labels.append(label_ids)
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# flat_true_labels = [item for sublist in true_labels for item in sublist]
print(flat_predictions[0])

sentences
['They drank the pub.']
tokenized_texts
[['[CLS]', 'they', 'drank', 'the', 'pub', '.', '[SEP]']]
0
